In [1]:
import sys
sys.path.insert(0,'..')
from gp3.inference.svi import MFSVI
from gp3.inference.laplace import Laplace
from gp3.likelihoods.likelihoods import Poisson, Bernoulli
from gp3.utils import data as sim
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
import GPy
import warnings
warnings.filterwarnings('ignore')
from tqdm import trange
import numpy as np
from scipy.special import expit

In [2]:
X = sim.sim_X_equispaced(D = 2, N_dim = 20, lower=0, upper=100)
f = sim.sim_f(X, k=GPy.kern.RBF(2, variance=1.0, lengthscale=40.), mu = 4.)
y = sim.poisson_draw(f, 1.0) 

trace_func = go.Scatter3d(x = X[:,0], y = X[:,1], z=f, mode = 'markers', marker=dict(size = 2,))
trace_draws = go.Scatter3d(x = X[:,0], y = X[:,1], z=y, mode = 'markers', marker=dict(size = 2,))
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_func, 1, 1)
fig.append_trace(trace_draws, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



In [3]:
mu = np.zeros([X.shape[0]])
inf_svi = MFSVI(GPy.kern.RBF(1, 1., 40.), Poisson(), X, y, mu, noise = 1e-2, verbose = True)

In [4]:
from gp3.utils.kron import kron_list

inf_svi.run(1000, n_samples = 1)

iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=inf_svi.predict(), mode = 'markers', marker=dict(size = 2,), name = 'SVI posterior mean')])
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=np.exp(inf_svi.q_S), mode = 'markers', marker=dict(size = 2,), name = 'SVI posterior mean')])

ELBO: -60937.52 | KL: 119368.29 | logL: 58430.77 | step: 0.0: 100%|██████████| 1000/1000 [00:09<00:00, 109.42it/s]         


In [5]:
iplot([go.Scatter(x = range(len(inf_svi.elbos)), y = inf_svi.elbos)])
iplot([go.Scatter(x = range(len(inf_svi.likes)), y = inf_svi.likes)])
iplot([go.Scatter(x = range(len(inf_svi.kls)), y = inf_svi.kls)])
iplot([go.Scatter(x = range(len(inf_svi.mu_grad_norms)), y = inf_svi.mu_grad_norms)])
iplot([go.Scatter(x = range(len(inf_svi.S_grad_norms)), y = inf_svi.S_grad_norms)])

In [6]:
X_part, y_part = sim.rand_partial_grid(X, y, 0.25)
X_full, y_full, obs_idx, imag_idx = sim.fill_grid(X_part, y_part)

color = np.zeros(X_full.shape[0])
color[obs_idx] = 1.0
trace_partial_obs = go.Scatter3d(x = X_full[obs_idx, 0], y = X_full[obs_idx, 1],
                                 z= y[obs_idx], mode = 'markers', marker=dict(size = 2))

inf_svi = MFSVI(GPy.kern.RBF(1, 1., 40.), Poisson(), X, y_part, mu, obs_idx = obs_idx, noise = 1e-2, max_grad = 1, verbose = True)
inf_svi.run(1000, n_samples = 1)
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=inf_svi.predict(), mode = 'markers', marker=dict(size = 2, color = color), name = 'SVI posterior mean')])
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=np.exp(inf_svi.q_S), mode = 'markers', marker=dict(size = 2, color = color, colorscale = "Reds"),
                    name = 'SVI posterior variance')])

ELBO: -7426.53 | KL: 24422.87 | logL: 16996.34 | step: 0.0: 100%|██████████| 1000/1000 [00:07<00:00, 133.09it/s]             


In [7]:
iplot([go.Scatter(x = range(len(inf_svi.elbos)), y = inf_svi.elbos)])
iplot([go.Scatter(x = range(len(inf_svi.likes)), y = inf_svi.likes)])
iplot([go.Scatter(x = range(len(inf_svi.kls)), y = inf_svi.kls)])
iplot([go.Scatter(x = range(len(inf_svi.mu_grad_norms)), y = inf_svi.mu_grad_norms)])
iplot([go.Scatter(x = range(len(inf_svi.S_grad_norms)), y = inf_svi.S_grad_norms)])

In [8]:
print np.linalg.norm(np.exp(inf_svi.q_S[inf_svi.obs_idx]))/len(obs_idx)
print np.linalg.norm(np.exp(inf_svi.q_S[~inf_svi.obs_idx]))/(inf_svi.n-len(obs_idx))

0.00123769134622
0.000197842541711
